# En local!! Ollama + llama2

In [ ]:
#conda create -n langchain python=3.12 -y
#conda activate langchain
#pip3 install torch torchvision
#pip3 install langchain langchain-community pypdf openai chromadb tiktoken sentence-transformers transformers ipykernel


In [ ]:
#uv venv
#".venv/Scipts/activate.bat"
# uv pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
#uv pip install langchain langchain-community pypdf openai chromadb tiktoken sentence-transformers transformers

In [1]:
import requests
from langchain_community.document_loaders import PyPDFLoader

urls = [
    'https://arxiv.org/pdf/2306.06031v1.pdf',
    'https://arxiv.org/pdf/2306.12156v1.pdf',
    'https://arxiv.org/pdf/2306.14289v1.pdf',
    'https://arxiv.org/pdf/2305.10973v1.pdf',
    'https://arxiv.org/pdf/2306.13643v1.pdf'
]

ml_papers = []

for i, url in enumerate(urls):
    response = requests.get(url)
    filename = f'paper{i+1}.pdf'
    with open(filename, 'wb') as f:
        f.write(response.content)
        print(f'Descargado {filename}')

        loader = PyPDFLoader(filename)
        data = loader.load()
        ml_papers.extend(data)

# Utiliza la lista ml_papers para acceder a los elementos de todos los documentos descargados
print('Contenido de ml_papers:')
print()

c:\Users\lftob\miniconda3\envs\langchain\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Descargado paper1.pdf
Descargado paper2.pdf
Descargado paper3.pdf
Descargado paper4.pdf
Descargado paper5.pdf
Contenido de ml_papers:



In [2]:
type(ml_papers), len(ml_papers), ml_papers[3]

(list,
 57,
 Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-06-12T00:32:18+00:00', 'author': '', 'keywords': '', 'moddate': '2023-06-12T00:32:18+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'templateversion': 'IJCAI.2023.0', 'title': '', 'trapped': '/False', 'source': 'paper1.pdf', 'total_pages': 7, 'page': 3, 'page_label': '4'}, page_content='Figure 1: FinGPT Framework.\n4.1 Data Sources\nThe first stage of the FinGPT pipeline involves the collec-\ntion of extensive financial data from a wide array of online\nsources. These include, but are not limited to:\n• Financial news: Websites such as Reuters, CNBC, Yahoo\nFinance, among others, are rich sources of financial news\nand market updates. These sites provide valuable informa-\ntion on market trends, company earnings, macroeconomic\nindicators, and other financial events.\n• Social media: Pl

# Split de documentos

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,  #Tamaño del texto de cada chunk
    chunk_overlap=200, #Hace que al principio de cada chunk esten 200 caracteres del anterior, para dar continuidad
    length_function=len #Se hace que el chunk sea por longitud
    # Hay una forma de evaluar los chunks https://chunkviz.up.railway.app/
    )

documents = text_splitter.split_documents(ml_papers)

In [4]:
len(documents), documents[10]

(211,
 Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-06-12T00:32:18+00:00', 'author': '', 'keywords': '', 'moddate': '2023-06-12T00:32:18+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'templateversion': 'IJCAI.2023.0', 'title': '', 'trapped': '/False', 'source': 'paper1.pdf', 'total_pages': 7, 'page': 2, 'page_label': '3'}, page_content='highly volatile, changing rapidly in response to news events\nor market movements.\nTrends, often observable through websites like Seeking\nAlpha, Google Trends, and other finance-oriented blogs and\nforums, offer critical insights into market movements and in-\nvestment strategies. They feature:\n• Analyst perspectives: These platforms provide access to\nmarket predictions and investment advice from seasoned\nfinancial analysts and experts.\n• Market sentiment: The discourse on these platforms can\nreflect th

# Embeddings e ingesta de datos vectorial

In [5]:
import os
hf_token = os.getenv("HUGGING_FACE_TOKEN")

In [6]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

chunks = text_splitter.split_documents(ml_papers) #documents

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-en-v1.5"
)

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="./chroma_db"
)
# Un retriever convierte la base de datos "retorna" los fragmentos clave para responder la pregunta
# retriever = vectorstore.as_retriever(
#     search_kwargs={"k": 3}
# )

# 2. Mejora el Retriever
# Aumentamos k a 5 para dar más contexto y usamos "mmr" (Maximum Marginal Relevance)
# para evitar fragmentos que digan lo mismo y obtener información diversa.
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5, "fetch_k": 10}
)

C:\Users\lftob\AppData\Local\Temp\ipykernel_17836\1419044909.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
c:\Users\lftob\miniconda3\envs\langchain\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\lftob\.cache\huggingface\hub\models--BAAI--bge-large-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, s

In [18]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOllama(
    model="qwen2.5:7b", #qwen3-vl:4b
    temperature=0
)

prompt = ChatPromptTemplate.from_template("""
Responde la pregunta usando exclusivamente el contexto.
Si no está en el contexto, di literalmente y únicamente "No lo sé" y nada más.
                                          

Contexto:
{context}

Pregunta:
{question}
""")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_chain = (
    {
        "context": retriever | format_docs,
        "question": lambda x: x
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [8]:
qa_chain.invoke("¿Qué es FinGPT?")

'FinGPT es un marco de trabajo abierto y gratuito para modelos de lenguaje grande especializados en finanzas (FinLLMs). Este marco adopta una abordaje centrado en los datos, enfatizando la importancia de la adquisición, limpieza y preprocesamiento de datos. FinGPT tiene como objetivo democratizar el acceso a los datos financieros y los modelos FinLLMs, promoviendo la investigación, colaboración e innovación en finanzas. Proporciona una serie de aplicaciones prácticas para tareas financieras, incluyendo asesoramiento robótico, trading cuantitativo, desarrollo de bajo código, detección de fraude financiero, calificación crediticia, predicción de insolvencia, pronóstico de M&A y evaluación ESG. Además, ofrece una interfaz amigable para el desarrollo de software y sirve como un tutor en educación financiera.'

In [9]:
qa_chain.invoke("Porqué es dificil entrenar FinGPT?")

'Es difícil entrenar FinGPT debido a los desafíos en la obtención de datos de calidad, el manejo de diversos formatos y tipos de datos, y la gestión de inconsistencias en la calidad de los datos. Especialmente, la extracción de datos históricos o especializados puede ser compleja debido a los medios de datos variados como plataformas web, APIs, documentos PDF y imágenes.'

In [20]:
qa_chain.invoke("qué es fast segment anything?")

'FastSAM es un método para la tarea de segmentación todo-en-un (segment anything), que se divide en dos etapas principales: la segmentación de todas las instancias y la selección guiada por el prompt. La primera etapa detecta y segmenta todos los objetos en la imagen, similar al proceso de segmentación panóptica. La segunda etapa utiliza los prompts proporcionados para separar los objetos específicos de interés del panorama segmentado. Este enfoque decupla la tarea, reduciendo su complejidad y permitiendo la propuesta de un modelo de segmentación todo-en-un en tiempo real.'

In [11]:
qa_chain.invoke("cuál es la diferencia entre fast sam y mobile sam?")

'FastSAM tiene 68M parámetros y tarda 40ms en procesar una imagen, mientras que MobileSAM tiene menos de 10M parámetros, lo cual es significativamente más pequeño, y solo toma 10ms para procesar una imagen, lo que es 4 veces más rápido que FastSAM. Además, el mIoU de MobileSAM es mucho mayor que el de FastSAM, lo que indica un mejor rendimiento en la predicción de máscaras.'

In [12]:
qa_chain.invoke("cuál es la temperatura aproximada de la superficie del sol?")

'No lo sé'